In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import import_ipynb
import bella
import cnn
import json

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_file(filename):
    with open(filename) as data_file:
        dataset_js = json.load(data_file)
        dataset = pd.DataFrame(dataset_js)
        data_file.close()
    x = np.array(dataset['data'])
    y = np.array(dataset['labels'])
    return x, y
x, y = load_file('./shipsnet.json')

In [19]:
with open('./shipsnet.json') as data_file:
    dataset_js = json.load(data_file)
    dataset = pd.DataFrame(dataset_js)



In [18]:
dataset_js = json.load(data_file)
dataset = pd.DataFrame(dataset_js)

NameError: name 'dataset' is not defined

In [17]:
x.astype('uint8')

ValueError: setting an array element with a sequence.

In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 80, 80, 3])


  # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

  # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
cnn_model_fn()

In [9]:
x.shape

(4000,)

In [14]:
x[0]

[82,
 89,
 91,
 87,
 89,
 87,
 86,
 86,
 86,
 86,
 84,
 86,
 90,
 95,
 87,
 86,
 80,
 81,
 86,
 88,
 87,
 87,
 88,
 88,
 86,
 86,
 88,
 82,
 85,
 90,
 90,
 89,
 81,
 92,
 89,
 83,
 83,
 90,
 87,
 80,
 85,
 88,
 86,
 81,
 89,
 87,
 89,
 91,
 93,
 89,
 86,
 91,
 90,
 90,
 89,
 91,
 92,
 92,
 92,
 87,
 85,
 92,
 90,
 92,
 87,
 85,
 91,
 85,
 83,
 92,
 96,
 91,
 82,
 81,
 88,
 92,
 91,
 89,
 84,
 83,
 89,
 91,
 89,
 88,
 87,
 87,
 87,
 88,
 88,
 84,
 84,
 76,
 67,
 90,
 87,
 84,
 82,
 88,
 96,
 86,
 87,
 86,
 86,
 88,
 91,
 90,
 85,
 82,
 88,
 88,
 80,
 82,
 71,
 80,
 84,
 82,
 81,
 89,
 87,
 86,
 88,
 87,
 88,
 86,
 96,
 98,
 92,
 88,
 91,
 92,
 84,
 87,
 91,
 87,
 81,
 91,
 84,
 86,
 90,
 87,
 83,
 84,
 80,
 78,
 79,
 85,
 96,
 88,
 90,
 91,
 86,
 87,
 90,
 90,
 89,
 87,
 92,
 101,
 87,
 87,
 91,
 93,
 86,
 85,
 87,
 88,
 89,
 90,
 88,
 86,
 89,
 91,
 83,
 94,
 88,
 84,
 83,
 84,
 87,
 86,
 86,
 85,
 85,
 87,
 90,
 89,
 82,
 84,
 88,
 86,
 82,
 80,
 77,
 77,
 81,
 91,
 84,
 86,
 86,
 91,

In [15]:
x.reshape([-1, 3, 80, 80])

ValueError: cannot reshape array of size 4000 into shape (3,80,80)